# Recommendation - Data Preparation 🎬

---

<img src="https://cdn-images-1.medium.com/max/1200/0*ePGWILY6GyplT-nn" />

---

In the next few challenges, you will build a powerful **movie recommender**.

We will use the open-source library [LightFM](https://github.com/lyst/lightfm) which provides easy python implementation of **hybrid** recommendation engines.

In this first part, we will prepare the data in order to train efficiently of the model.

We let you load the data `movies` and `ratings` downloaded from the **small** [movielens dataset](https://grouplens.org/datasets/movielens/).



In [11]:
pip install lightfm

Note: you may need to restart the kernel to use updated packages.


In [43]:
### TODO: Load the movies and ratings datasets

import pandas as pd
movies = pd.read_csv("/Users/sunwoonam/Desktop/Griffith/2023_T1/7130ICT_Data Analytics/Lab/Lab10/notebooks/ml-latest-small/movies.csv")
ratings = pd.read_csv("/Users/sunwoonam/Desktop/Griffith/2023_T1/7130ICT_Data Analytics/Lab/Lab10/notebooks/ml-latest-small/ratings.csv")

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


**Q1**. What are the different types of recommendation models? Explain briefly with your own words the differences between them.

In [45]:
print("1. Content-based recommendation: Recommend based on the characteristics or attributes of the items themselves.")
print("2. Rating-based recommendation (Collaborative Filtering): Recommend based on the rating matrix.")
print("3. Clustering-based recommendation: Recommend based on ratings of each cluster.")
print("4. Hybrid Recommendation: Combine multiple recommendation techniques. These models can leverage both collaborative filtering and content-based filtering approaches.")

1. Content-based recommendation: Recommend based on the characteristics or attributes of the items themselves.
2. Rating-based recommendation (Collaborative Filtering): Recommend based on the rating matrix.
3. Clustering-based recommendation: Recommend based on ratings of each cluster.
4. Hybrid Recommendation: Combine multiple recommendation techniques. These models can leverage both collaborative filtering and content-based filtering approaches.


**Q1bis**. What data is expected by the LightFM `fit` method? Especially, how does the train data should be organized, and what should be the type of the train dataset? 

In [16]:
print("training data is a (no_users, no_items) sparses mtarix (with 1s donoting positive, and -1s negative interactions)")

training data is a (no_users, no_items) sparses mtarix (with 1s donoting positive, and -1s negative interactions)


**Q2**. Explore `movies` and `ratings`, what do those datasets contain? How are they organized?

In [18]:
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
print(ratings.shape)
print(ratings.head())

len(ratings['userId'].unique())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


610

---

### Q3 & Q4 are optional
> you can come back to it if you have time after having finished the whole project of the day

We created a few utils functions for you in `utils.py` script. Especially:
- `threshold_interactions_df`:
> Limit interactions df to minimum row and column interactions

**Q3**. Open `src/utils.py` file, and have a look at the documentation of this function to understand its goal and how it works.

Have a look the code to understand fully how it works. You should be familiar with everything.

What does represent the variable `sparsity`? What is the range of values in which sparsity can be?

In [47]:
file_path = '/Users/sunwoonam/Desktop/Griffith/2023_T1/7130ICT_Data Analytics/Lab/Lab10/notebooks/utils.py'

try:
    with open(file_path, 'r') as file:
        file_contents = file.read()
        print(file_contents)
except FileNotFoundError:
    print("File not found.")

SyntaxError: incomplete input (795987394.py, line 6)

**Q4**. Create a new DataFrame `ratings_thresh`, that filters `ratings` with only:
- users that rated strictly more than 4 movies
- movies that have been rated at least 10 times

How many users/movies remain in this new dataset?

In [32]:
from utils import threshold_interactions_df
ratings_thresh = threshold_interactions_df(ratings, 'userId','movieId',5,10)
print("With these new conditions, it remains {} users and {} movies.".format(    
    len(ratings_thresh.userId.unique()),
    len(ratings_thresh.movieId.unique())))

Starting interactions info
Number of rows: 610
Number of cols: 9724
Sparsity: 1.700%
Ending interactions info
Number of rows: 610
Number of columns: 3650
Sparsity: 4.055%
With these new conditions, it remains 610 users and 3650 movies.


In [33]:
ratings_thresh.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


**Q5**. In order to fit a [LightFM](https://lyst.github.io/lightfm/docs/home.html) model, we need to transform our Dataframe to a sparse matrix (cf. below). This is not straightforward so we included the function `df_to_matrix` in `utils.py`.

> 🔦 **Hint**:  Sparse matrices are just **big matrices with a lot of zeros or empty values**.
> 
> Existing tools (Pandas DataFrame, Numpy arrays for example) are not suitable for manipulating this kind of data. So we will use [Scipy sparse matrices](https://docs.scipy.org/doc/scipy-0.14.0/reference/sparse.html).
>
> It exists many different "types" of sparse matrices (CSC, CSR, COO, DIA, etc.). You don't need to know them. Just know that it corresponds to different formats with different methods of manipulation, slicing, indexing, etc.

> 🔦 **Hint 2**:  By going from a DataFrame to a sparse matrix, you will lose the information of the ids (userId and movieId), you will only deal with indices (row number and column number). Therefore, the `df_to_matrix` function also returns dictionaries mapping indexes to ids (ex: uid_to_idx mapping userId to index of the matrix) 


Have a look at the util function documentation, and use it to create 5 new variables:
- a final sparse matrix `ratings_matrix` (this will be the data used to train the model)
- the following utils mappers:
    - `uid_to_idx`
    - `idx_to_uid`
    - `mid_to_idx`
    - `idx_to_mid`

In [35]:
from utils import df_to_matrix

ratings_matrix, uid_to_idx, idx_to_uid, mid_to_idx, idx_to_mid = df_to_matrix(ratings, 'userId','movieId')
ratings_matrix

<610x9724 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

**Q6**.
- On the one side, find what movies did the userId 4 rate?

- On the other side, what is the value of `ratings_matrix` for:
    - userId = 4 and movieId=1
    - userId = 4 and movieId=2
    - userId = 4 and movieId=21
    - userId = 4 and movieId=32
    - userId = 4 and movieId=126

Conclude on the values signification in `ratings_matrix`

In [36]:
ratings[ratings.userId==4]

,userId,movieId,rating,timestamp
300,4,21,3.0,986935199
301,4,32,2.0,945173447
302,4,45,3.0,986935047
303,4,47,2.0,945173425
304,4,52,3.0,964622786
...,...,...,...,...
511,4,4765,5.0,1007569445
512,4,4881,3.0,1007569445
513,4,4896,4.0,1007574532
514,4,4902,4.0,1007569465


In [37]:
for mid in [1,2,21,32,126]:
    print("For mid={}".format(mid))
    print("Value of ratings_matrix={}".format(
        ratings_matrix[uid_to_idx[4], mid_to_idx[mid]]))
print("ratings matrix has value equal to when user rated movie and 0 otherwise")


For mid=1
Value of ratings_matrix=0.0
For mid=2
Value of ratings_matrix=0.0
For mid=21
Value of ratings_matrix=1.0
For mid=32
Value of ratings_matrix=1.0
For mid=126
Value of ratings_matrix=1.0
ratings matrix has value equal to when user rated movie and 0 otherwise


**Q5**. Now that you have a `ratings_matrix` in the correct format, let's save it in pickle format:
- Create a variable `dst_dir` corresponding to the path of the folder `data/netflix` located at the root of the repository
- **Verify that this is the correct path**
- Save the ratings_matrix in pickle (as `ratings_matrix.pkl`) in this corresponding directory

In [39]:
dst_dir = "./data"
import pickle
pickle.dump(ratings_matrix,open(dst_dir + "/ratings_matrix.pkl","wb"))

FileNotFoundError: [Errno 2] No such file or directory: './data/ratings_matrix.pkl'

**Q6**. Save also all mappings objects into pickle (`idx_to_mid`, `mid_to_idx`, `uid_to_idx`, `idx_to_uid`) as it will be useful for later.

In [40]:
pickle.dump(idx_to_mid, open(dst_dir + "/idx_to_mid.pkl","wb"))
pickle.dump(mid_to_idx, open(dst_dir + "/mid_to_idx.pkl","wb"))
pickle.dump(uid_to_idx, open(dst_dir + "/uid_to_idx.pkl","wb"))
pickle.dump(idx_to_uid, open(dst_dir + "/idx_to_uid.pkl","wb"))

FileNotFoundError: [Errno 2] No such file or directory: './data/idx_to_mid.pkl'

In [41]:
pickle.dump(movies, open(dst_dir + "/movies.pk1","wb"))

FileNotFoundError: [Errno 2] No such file or directory: './data/movies.pk1'

Up to next challenge now! 🍿